In [1]:
import os
from glob import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter


import pickle
import gc

from tqdm import tqdm
tqdm.pandas()

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, make_scorer

In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb

In [4]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = './models'
UTILS = './utils'

## load data, features

In [38]:
load_dtypes = pickle.load(open(os.path.join(UTILS, 'load_dtypes.pkl'), 'rb'))

In [39]:
using_features = pickle.load(open(os.path.join(DATA_OWN, 'using_features.pkl'), 'rb'))
len(using_features)

91

In [40]:
load_col = list(using_features)
load_col.extend(['client_pin', 'timestamp', 'multi_class_target'])

In [41]:
%%time
data_train = pd.read_csv(os.path.join(DATA_OWN, 'data_train.csv'), parse_dates=['timestamp'], usecols=load_col, dtype=load_dtypes)
#data_train = pd.read_csv(os.path.join(DATA_OWN, 'data_train.csv'), usecols=load_col, dtype=load_dtypes)
data_train.head()

Wall time: 1min 25s


,client_pin,timestamp,multi_class_target,client_freq_main_screen,client_freq_statement,client_freq_credit_info,client_freq_own_transfer,client_freq_mobile_recharge,client_freq_phone_money_transfer,client_freq_card2card_transfer,...,relations_time_past_targ_main_screen,relations_time_past_targ_statement,relations_time_past_targ_credit_info,relations_time_past_targ_own_transfer,relations_time_past_targ_mobile_recharge,relations_time_past_targ_phone_money_transfer,relations_time_past_targ_card2card_transfer,relations_time_past_targ_chat,relations_time_past_targ_card_recharge,relations_time_past_targ_invest
0,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen,0.133333,0.380952,0.000000,0.000000,0.066667,0.142857,0.276190,...,1.307955,0.163198,0.000000,0.000000,1.046685,0.341562,1.533734,0.000000,0.000000,0.000000
1,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest,0.389706,0.573529,0.000000,0.007353,0.000000,0.000000,0.000000,...,0.032518,2.121950,0.000000,0.706016,0.000000,0.000000,0.000000,1.837007,0.000000,1.374358
2,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement,0.116279,0.465116,0.000000,0.046512,0.279070,0.069767,0.000000,...,0.280475,0.009147,0.000000,0.421385,1.205900,0.271085,0.000000,0.234015,0.000000,0.000000
3,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen,0.457831,0.449799,0.000000,0.004016,0.032129,0.008032,0.024096,...,6.314792,0.452830,0.000000,0.575543,0.363213,1.093542,0.614683,0.580554,0.530097,1.346742
4,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen,0.600000,0.066667,0.266667,0.000000,0.000000,0.000000,0.000000,...,0.107516,0.446540,0.446022,0.000000,0.000000,0.000000,0.000000,0.000000,0.371353,0.000000


In [42]:
(data_train.isnull().values.any())

False

In [10]:
target = data_train.multi_class_target
data_train.drop('multi_class_target', axis = 1, inplace = True)

In [11]:
%%time
X_train, X_val, y_train, y_val = train_test_split(data_train, target, test_size=0.33, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

Wall time: 7.64 s


((3393784, 91), (1671566, 91), (3393784,), (1671566,))

In [19]:
#using_features = pickle.load(open(os.path.join(DATA_OWN, 'using_features.pkl'), 'rb'))
#len(using_features)

In [20]:
#f1 = make_scorer(f1_score , average='macro')

## check classifiers with check time of work

In [21]:
#clf_sgd_log   = SGDClassifier(loss = 'log', class_weight='balanced', n_jobs=-1)

#clf_knn  = KNeighborsClassifier()
#clf_svc  = SVC()
#clf_gaus = GaussianNB()

In [16]:
%%time
#clf_sgd_hinge = SGDClassifier(loss = 'hinge', class_weight='balanced', n_jobs=-1)
clf_sgd_hinge = SGDClassifier(loss = 'hinge', n_jobs=-1)
clf_sgd_hinge.fit(X_train[using_features], y_train)
pred_sgd_hinge = clf_sgd_hinge.predict(X_val[using_features])
print(len(set(pred_sgd_hinge)), set(pred_sgd_hinge))
print(f1_score(y_val, pred_sgd_hinge, average  = 'micro'))

10 {'phone_money_transfer', 'invest', 'own_transfer', 'card2card_transfer', 'mobile_recharge', 'chat', 'credit_info', 'main_screen', 'card_recharge', 'statement'}
0.6503009752531459
Wall time: 59.1 s


In [17]:
#%%time
#clf_sgd_perc  = SGDClassifier(loss = 'perceptron', class_weight='balanced', n_jobs=-1)
#clf_sgd_perc  = SGDClassifier(loss = 'perceptron', n_jobs=-1)
#clf_sgd_perc.fit(X_train[using_features], y_train)
#pred_sgd_perc = clf_sgd_perc.predict(X_val[using_features])
#rint(len(set(pred_sgd_perc)), set(pred_sgd_perc))
#rint(f1_score(y_val, pred_sgd_perc, average  = 'micro'))

In [18]:
%%time
# 9 min
# 91 - 1h 10min 53s

clf_mlp  = MLPClassifier((200, 50), learning_rate = 'adaptive', activation='logistic', verbose = True)
clf_mlp.fit(X_train[using_features], y_train)
pred_mlp = clf_mlp.predict(X_val[using_features])
print(len(set(pred_mlp)), set(pred_mlp))
print(f1_score(y_val, pred_mlp, average  = 'micro'))

Iteration 1, loss = 0.94448429
Iteration 2, loss = 0.87399526
Iteration 3, loss = 0.86408264
Iteration 4, loss = 0.85787957
Iteration 5, loss = 0.85306122
Iteration 6, loss = 0.84875746
Iteration 7, loss = 0.84491059
Iteration 8, loss = 0.84147376
Iteration 9, loss = 0.83864808
Iteration 10, loss = 0.83615396
Iteration 11, loss = 0.83394565
Iteration 12, loss = 0.83206609
Iteration 13, loss = 0.83040759
Iteration 14, loss = 0.82893533
Iteration 15, loss = 0.82763676
Iteration 16, loss = 0.82645360
Iteration 17, loss = 0.82535177
Iteration 18, loss = 0.82447853
Iteration 19, loss = 0.82372994
Iteration 20, loss = 0.82303128
Iteration 21, loss = 0.82235561
Iteration 22, loss = 0.82177943
Iteration 23, loss = 0.82116264
Iteration 24, loss = 0.82054930
Iteration 25, loss = 0.82013040
Iteration 26, loss = 0.81951289
Iteration 27, loss = 0.81911890
Iteration 28, loss = 0.81871800
Iteration 29, loss = 0.81819553
Iteration 30, loss = 0.81779440
Iteration 31, loss = 0.81735075
Iteration 32, los

In [19]:
%%time
# 81 - 8min 23s
# 91 - 8min 40s

clf_lr = LogisticRegression(n_jobs = -1, verbose = 1) #‘liblinear’,
clf_lr.fit(X_train[using_features], y_train)
pred_lr = clf_lr.predict(X_val[using_features])
print(len(set(pred_lr)), set(pred_lr))
print(f1_score(y_val, pred_lr, average  = 'micro'))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  8.3min finished


10 {'phone_money_transfer', 'invest', 'own_transfer', 'card2card_transfer', 'mobile_recharge', 'chat', 'credit_info', 'main_screen', 'card_recharge', 'statement'}
0.6548661554494408
Wall time: 8min 40s


In [20]:
%%time
# 81 - 17min 29s
# 91 - 4h 4min 35s
clf_lrsvc = LinearSVC(verbose = 1) # loss = ‘hinge’
clf_lrsvc.fit(X_train[using_features], y_train)
pred_lrsvc = clf_lrsvc.predict(X_val[using_features])
print(len(set(pred_lrsvc)), set(pred_lrsvc))
print(f1_score(y_val, pred_lrsvc, average  = 'micro'))

[LibLinear]

C:\Users\_v010ch_\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


10 {'phone_money_transfer', 'invest', 'own_transfer', 'card2card_transfer', 'mobile_recharge', 'chat', 'credit_info', 'main_screen', 'card_recharge', 'statement'}
0.6572238248444872
Wall time: 4h 4min 35s


In [21]:
%%time 
# 81 - 5min 38s
# 91 - 5min 21s

clf_lgbm = LGBMClassifier( silent = False) # loss = ‘hinge’
clf_lgbm.fit(X_train[using_features], y_train)
pred_lgbm = clf_lgbm.predict(X_val[using_features])
print(len(set(pred_lgbm)), set(pred_lgbm))
print(f1_score(y_val, pred_lgbm, average  = 'micro'))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.320528 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10921
[LightGBM] [Info] Number of data points in the train set: 3393784, number of used features: 91
[LightGBM] [Info] Start training from score -3.265595
[LightGBM] [Info] Start training from score -3.600559
[LightGBM] [Info] Start training from score -3.312343
[LightGBM] [Info] Start training from score -2.318198
[LightGBM] [Info] Start training from score -4.486080
[LightGBM] [Info] Start training from score -0.798248
[LightGBM] [Info] Start training from score -2.944099
[LightGBM] [Info] Start training from score -2.858770
[LightGBM] [Info] Start training from score -3.077214
[LightGBM] [Info] Start training from score -1.702772
10 {'phone_money_transfer', 'invest', 'own_transfer', 'mobile_recharge', 'chat', 'credit_info', 'main

In [ ]:
#%%time
#clf_cb = CatBoostClassifier()
#clf_cb.fit(X_train[using_features], y_train)
#pred_cb = clf_cb.predict(X_val[using_features])
#print(set(pred_cb))
#print(f1_score(y_val, pred_cb, average  = 'micro'))

### saw how many give us stack

In [41]:
%%time
pred_stack = [''] * pred_sgd_hinge.shape[0]
collisions = 0
for idx in range(pred_sgd_hinge.shape[0]):
    cnt = Counter([
                   pred_sgd_hinge[idx], 
                   #pred_mlp[idx], 
                   pred_lr[idx], 
                   pred_lrsvc[idx]
               ]).most_common()
    pred_stack[idx] = cnt[0][0]
    
    if len(cnt) > 1 and cnt[0][1] == cnt[1][1]:
        collisions += 1
    #    print('terrible!')
    #    print(cnt[0], cnt[1])

Wall time: 5.4 s


In [42]:
print(f1_score(y_val, pred_stack, average  = 'micro'), (collisions/pred_sgd_hinge.shape[0]))

0.6567954839952476


In [48]:
stack_df = pd.DataFrame({'SGD':pred_sgd_hinge, 'logit':pred_lr, 'linear_svc':pred_lrsvc})
cat_ftrs = ['SGD', 'logit', 'linear_svc']

In [54]:
#cb_stack = CatBoostClassifier(loss_function='Logloss', cat_features=cat_ftrs)
cb_stack = CatBoostClassifier(cat_features=cat_ftrs)
cb_stack.fit(stack_df, y_val)
pred_stack_cb = cb_stack.predict(X_val[using_features])
print(len(set(pred_stack_cb)), set(pred_stack_cb))
print(f1_score(y_val, pred_stack_cb, average  = 'micro'))

Learning rate set to 0.115532
0:	learn: 1.9487634	total: 10.7s	remaining: 2h 57m 45s
1:	learn: 1.7927731	total: 22.6s	remaining: 3h 7m 45s
2:	learn: 1.6929532	total: 33.1s	remaining: 3h 3m 30s
3:	learn: 1.6223091	total: 43.4s	remaining: 3h 14s
4:	learn: 1.5468016	total: 55.6s	remaining: 3h 4m 24s
5:	learn: 1.4827684	total: 1m 9s	remaining: 3h 10m 42s
6:	learn: 1.4361012	total: 1m 23s	remaining: 3h 16m 34s
7:	learn: 1.4021923	total: 1m 39s	remaining: 3h 25m 56s
8:	learn: 1.3740723	total: 1m 56s	remaining: 3h 33m 5s
9:	learn: 1.3522151	total: 2m 11s	remaining: 3h 37m 47s
10:	learn: 1.3353209	total: 2m 27s	remaining: 3h 41m 33s
11:	learn: 1.3212102	total: 2m 43s	remaining: 3h 44m 51s
12:	learn: 1.3096953	total: 2m 59s	remaining: 3h 47m 43s
13:	learn: 1.2996602	total: 3m 15s	remaining: 3h 49m 49s
14:	learn: 1.2912930	total: 3m 31s	remaining: 3h 51m 22s
15:	learn: 1.2831218	total: 3m 47s	remaining: 3h 53m 10s
16:	learn: 1.2762077	total: 4m 3s	remaining: 3h 54m 37s
17:	learn: 1.2709512	total

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=0]=0.15833333134651184 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [59]:
pred_stack_cb = cb_stack.predict(stack_df)
print(len(set(pred_stack_cb[:, 0])), set(pred_stack_cb[:, 0]))
print(f1_score(y_val, pred_stack_cb[:, 0], average  = 'micro'))

10 {'phone_money_transfer', 'statement', 'credit_info', 'invest', 'own_transfer', 'chat', 'mobile_recharge', 'card2card_transfer', 'card_recharge', 'main_screen'}
0.6571460534612453


In [ ]:
#dataset = cb.Pool(X, y, cat_features=['d'])

## study best models on full data

In [22]:
%%time
# 91 - 45min 1s ????

clf_sgd_hinge = SGDClassifier(loss = 'hinge', n_jobs=-1)
clf_sgd_hinge.fit(data_train[using_features], target)

Wall time: 45min 1s


SGDClassifier(n_jobs=-1)

In [23]:
%%time
# 91 - 1h 28min 38s
clf_mlp  = MLPClassifier((200, 50), learning_rate = 'adaptive', activation='logistic', verbose = True)
clf_mlp.fit(data_train[using_features], target)

Iteration 1, loss = 0.92197328
Iteration 2, loss = 0.86741594
Iteration 3, loss = 0.85780875
Iteration 4, loss = 0.85015409
Iteration 5, loss = 0.84372913
Iteration 6, loss = 0.83864466
Iteration 7, loss = 0.83476808
Iteration 8, loss = 0.83184363
Iteration 9, loss = 0.82930029
Iteration 10, loss = 0.82724432
Iteration 11, loss = 0.82560988
Iteration 12, loss = 0.82426798
Iteration 13, loss = 0.82308407
Iteration 14, loss = 0.82194414
Iteration 15, loss = 0.82103730
Iteration 16, loss = 0.82015614
Iteration 17, loss = 0.81938284
Iteration 18, loss = 0.81865858
Iteration 19, loss = 0.81799958
Iteration 20, loss = 0.81737564
Iteration 21, loss = 0.81679260
Iteration 22, loss = 0.81617593
Iteration 23, loss = 0.81571605
Iteration 24, loss = 0.81518562
Iteration 25, loss = 0.81486809
Iteration 26, loss = 0.81449454
Iteration 27, loss = 0.81420767
Iteration 28, loss = 0.81385686
Iteration 29, loss = 0.81361204
Iteration 30, loss = 0.81332871
Iteration 31, loss = 0.81307522
Iteration 32, los

MLPClassifier(activation='logistic', hidden_layer_sizes=(200, 50),
              learning_rate='adaptive', verbose=True)

In [25]:
%%time
# 81 - 13min 29s
# 91 - 1h 13min 58s
clf_lr = LogisticRegression(n_jobs = -1, verbose = 2)
clf_lr.fit(data_train[using_features], target)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 66.1min finished


Wall time: 1h 13min 58s


LogisticRegression(n_jobs=-1, verbose=2)

In [ ]:
%%time
#81 - 27min 9s

#clf_lrsvc = LinearSVC(verbose = True) # loss = ‘hinge’
#clf_lrsvc.fit(data_train[using_features], target)

In [26]:
clf_lgbm = LGBMClassifier( silent = False) # loss = ‘hinge’
clf_lgbm.fit(data_train[using_features], target)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.544585 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10921
[LightGBM] [Info] Number of data points in the train set: 5065350, number of used features: 91
[LightGBM] [Info] Start training from score -3.265532
[LightGBM] [Info] Start training from score -3.598471
[LightGBM] [Info] Start training from score -3.311040
[LightGBM] [Info] Start training from score -2.318178
[LightGBM] [Info] Start training from score -4.485760
[LightGBM] [Info] Start training from score -0.797913
[LightGBM] [Info] Start training from score -2.944861
[LightGBM] [Info] Start training from score -2.860032
[LightGBM] [Info] Start training from score -3.079522
[LightGBM] [Info] Start training from score -1.703016


LGBMClassifier(silent=False)

## save models

In [27]:
%%time
pickle.dump(clf_sgd_hinge, open(os.path.join(MODELS, 'clf_sgd.pkl'), 'wb'))
pickle.dump(clf_mlp, open(os.path.join(MODELS, 'clf_mlp.pkl'), 'wb'))
pickle.dump(clf_lr, open(os.path.join(MODELS, 'clf_lr.pkl'), 'wb'))
#pickle.dump(clf_lrsvc, open(os.path.join(MODELS, 'clf_lrsvc.pkl'), 'wb'))
#pickle.dump(clf_svc, open(os.path.join(MODELS, 'clf_svc.pkl'), 'wb'))
#pickle.dump(clf_rf,  open(os.path.join(MODELS, 'clf_rf.pkl'),  'wb'))
            
#clf_cb.save_model(os.path.join(MODELS, 'clf_cb.cbm'), format='cbm')
#lgb.save(clf_lgbm, os.path.join(MODELS, 'clf_lgbm.lgb'), num_iteration = NULL)
#clf_lgbm.save_model(os.path.join(MODELS, 'clf_lgbm.txt'))

pickle.dump(clf_lgbm, open(os.path.join(MODELS, 'clf_lgbm.pkl'), 'wb'))

Wall time: 443 ms
